In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords


print("All imports successful!")


In [ ]:
import pandas as pd

# Load CSV file
df = pd.read_csv("IMDB Dataset.csv")

# Preview first 5 rows
df.head(10)


In [ ]:
# Check dataset info
df.info()

# Check for missing values
df.isnull().sum()

# Check sentiment distribution
df['sentiment'].value_counts()

df.isnull().sum()


In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
import re 
from nltk.corpus import stopwords
sample_review = df['review'][0]
cleaned_review = sample_review.lower()
cleaner_review = re.sub( r'[^a-z\s]' , '', cleaned_review)
stop_words = set(stopwords.words('english'))
words = cleaner_review.split()
words = [w for w in words if w not in stop_words]
cleaned_text = " ".join(words)
print(cleaned_text)

In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def cleaned_text(text):
    cleaner_review = re.sub( r'[^a-z\s]' , '', text.lower())
    words = cleaner_review.split()
    words = [w for w in words if w not in stop_words]
    cleaned_text = " ".join(words)
    return cleaned_text
df = pd.read_csv("IMDB Dataset.csv")
df['clean_review'] = df['review'].apply(cleaned_text)
df.head()


In [ ]:
def label(sentiment):
    if sentiment == 'positive' :
        return 1
    else:
        return 0

df = pd.read_csv("IMDB Dataset.csv")
df['sentiment_label'] = df['sentiment'].apply(label)
df['sentiment_label'].value_counts()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


df = pd.read_csv("IMDB Dataset.csv")
df['clean_review'] = df['review'].apply(cleaned_text)
df['sentiment_label'] = df['sentiment'].apply(label)
X = df['clean_review']
y = df['sentiment_label']
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    shuffle=True,
    stratify=y
)

vectorizer = TfidfVectorizer(max_features=5000)  # Keep top 5000 words
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)
print(X_train_vectors.shape)  # rows = number of reviews, columns = number of features
print(X_test_vectors.shape)




In [ ]:
from sklearn.linear_model import LogisticRegression        
from sklearn.metrics import accuracy_score, classification_report  
import joblib

model = LogisticRegression(max_iter=1000)  
model.fit(X_train_vectors, y_train)
y_pred = model.predict(X_test_vectors)
# Overall accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

# Detailed metrics
print("Classification Report:\n", classification_report(y_test, y_pred))

joblib.dump(model, 'sentiment_model.pkl')

# Save vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')



In [ ]:
import re
from nltk.corpus import stopwords
import joblib

def predict_sentiment(review):
    # Load saved model and vectorizer
    model = joblib.load('sentiment_model.pkl')
    vectorizer = joblib.load('tfidf_vectorizer.pkl')
    
    # Clean the review
    cleaned_review = re.sub(r'[^a-z\s]', '', review.lower())
    stop_words = set(stopwords.words('english'))
    words = cleaned_review.split()
    words = [w for w in words if w not in stop_words]
    cleaned_text = " ".join(words)
    
    # Convert review to numeric vector using the loaded vectorizer
    X_new = vectorizer.transform([cleaned_text])  # Note: transform, not fit_transform
    
    # Make prediction
    prediction = model.predict(X_new)
    
    # Return readable output
    return "Positive" if prediction[0] == 1 else "Negative"



review = "The acting was mediocre, but the story kept me hooked till the end."
print(predict_sentiment(review))


    

   
    


